In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3

## Store CSV into DataFrame, Extract & Transform
**We have five csv file:**
* Airlines - has information about airline_id(IATA Code) and airline name;
* Airports - has information about airport_id, airport, city, state, country, latitude, longitude
* Flights  - contains information about date, day of the week,airline id, flight number,tail number,origin airport, destination airport, scheduled departure, actual departure time,delay(in minutes), scheduled arrival, Arrival delay, and cancellation.
* tweets   - information about tweets regarding the flight experience,ariline and date of tweets.
* there is one additional file: cancellation reason. 

# 1. Airline

In [2]:
airline_file = "./Resources/airlines.csv"
airline_df = pd.read_csv(airline_file)
airline_df

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [3]:
airline_df.columns = ['AIRLINE_ID','AIRLINE']
# airline_df.set_index('AIRLINE_ID',inplace=True)
airline_df.head(5)

,AIRLINE_ID,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


# 2. Airport

In [4]:
airports_file = "./Resources/airports.csv"
airports_df = pd.read_csv(airports_file)
airports_df.columns =['AIRPORT_ID','AIRPORT','CITY','STATE','COUNTRY','LATITUDE','LONGITUDE']
# airports_df.set_index('AIRPORT_ID',inplace=True)
airports_df.head()

,AIRPORT_ID,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


# 3. Flight

In [5]:
flights_file = "./Resources/flights.csv"
flights_df = pd.read_csv(flights_file)
flights_df.head(5)

C:\Users\DELGCA01\anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Check Datatypes of all the columns

In [6]:
flights_df.dtypes

YEAR                     int64
MONTH                    int64
DAY                      int64
DAY_OF_WEEK              int64
AIRLINE                 object
FLIGHT_NUMBER            int64
TAIL_NUMBER             object
ORIGIN_AIRPORT          object
DESTINATION_AIRPORT     object
SCHEDULED_DEPARTURE      int64
DEPARTURE_TIME         float64
DEPARTURE_DELAY        float64
TAXI_OUT               float64
WHEELS_OFF             float64
SCHEDULED_TIME         float64
ELAPSED_TIME           float64
AIR_TIME               float64
DISTANCE                 int64
WHEELS_ON              float64
TAXI_IN                float64
SCHEDULED_ARRIVAL        int64
ARRIVAL_TIME           float64
ARRIVAL_DELAY          float64
DIVERTED                 int64
CANCELLED                int64
CANCELLATION_REASON     object
AIR_SYSTEM_DELAY       float64
SECURITY_DELAY         float64
AIRLINE_DELAY          float64
LATE_AIRCRAFT_DELAY    float64
WEATHER_DELAY          float64
dtype: object

## Subset the airlines matching the tweets data

In [7]:
flights_df = flights_df.loc[(flights_df['AIRLINE']=='UA')| (flights_df['AIRLINE']=='US')| (flights_df['AIRLINE']=='AA')
                            | (flights_df['AIRLINE']=='WN') | (flights_df['AIRLINE']=='VX')| (flights_df['AIRLINE']=='DL')]

In [8]:
flights_df.groupby('AIRLINE').count()

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
AIRLINE,,,,,,,,,,,,,,,,,,,,,
AA,725984,725984,725984,725984,725984,723673,725984,725984,725984,715598,...,714855,712935,725984,725984,10919,130279,130279,130279,130279,130279
DL,875881,875881,875881,875881,875881,875855,875881,875881,875881,872177,...,871946,870275,875881,875881,3824,118023,118023,118023,118023,118023
UA,515723,515723,515723,515723,515723,509556,515723,515723,515723,509534,...,508916,507762,515723,515723,6573,104722,104722,104722,104722,104722
US,198715,198715,198715,198715,198715,194825,198715,198715,198715,194825,...,194586,194223,198715,198715,4067,36549,36549,36549,36549,36549
VX,61903,61903,61903,61903,61903,61903,61903,61903,61903,61385,...,61365,61248,61903,61903,534,11778,11778,11778,11778,11778
WN,1261855,1261855,1261855,1261855,1261855,1260450,1261855,1261855,1261855,1246129,...,1245053,1242403,1261855,1261855,16043,236626,236626,236626,236626,236626


### Check NaN values

In [9]:
# Drop rows of NaN
# There isn't a row that's completely NaN
all_nan_df = flights_df[flights_df.isna().all(axis=1)]
all_nan_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY


In [10]:
# However, for the flights that's not delayed/ cancelled, those columns are NaN
any_nan_df = flights_df[flights_df.isna().any(axis=1)]
any_nan_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,1,1,4,DL,806,N3730B,SFO,MSP,25,...,610.0,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,1,1,4,US,2013,N584UW,LAX,CLT,30,...,753.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# For readibility, we replace NaN with 0 
flights_df = flights_df.fillna(0)
flights_df[flights_df.isna().any(axis=1)]

# For excluding 1 column: any_nan_df.loc[:, any_nan_df.columns != 'CANCELLATION_REASON'].fillna(0)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY


### Modify Columns and Labels of categories

In [12]:
# DROP DELAY columns
flights_df = flights_df.drop(columns=['AIR_SYSTEM_DELAY', 'SECURITY_DELAY','AIRLINE_DELAY',
                         'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])
flights_df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,0
5,2015,1,1,4,DL,806,N3730B,SFO,MSP,25,...,206.0,1589,604.0,6.0,602,610.0,8.0,0,0,0
7,2015,1,1,4,US,2013,N584UW,LAX,CLT,30,...,228.0,2125,745.0,8.0,803,753.0,-10.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819064,2015,12,31,4,UA,668,N66848,LAS,IAD,2359,...,203.0,2065,634.0,18.0,721,652.0,-29.0,0,0,0
5819065,2015,12,31,4,UA,671,N33209,PDX,IAH,2359,...,214.0,1825,536.0,9.0,604,545.0,-19.0,0,0,0
5819066,2015,12,31,4,UA,1849,N497UA,LAS,EWR,2359,...,231.0,2227,703.0,15.0,742,718.0,-24.0,0,0,0
5819067,2015,12,31,4,AA,2342,N3DLAA,DEN,MIA,2359,...,190.0,1709,519.0,6.0,543,525.0,-18.0,0,0,0


In [13]:
# Merge DateTime to one column
flights_df['DATE'] = pd.to_datetime(flights_df[['YEAR','MONTH','DAY']])
flights_df = flights_df.drop(columns=['YEAR','MONTH','DAY'])
flights_df.head(5)

,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,DATE
1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,...,2330,737.0,4.0,750,741.0,-9.0,0,0,0,2015-01-01
2,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,...,2296,800.0,11.0,806,811.0,5.0,0,0,0,2015-01-01
3,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,...,2342,748.0,8.0,805,756.0,-9.0,0,0,0,2015-01-01
5,4,DL,806,N3730B,SFO,MSP,25,20.0,-5.0,18.0,...,1589,604.0,6.0,602,610.0,8.0,0,0,0,2015-01-01
7,4,US,2013,N584UW,LAX,CLT,30,44.0,14.0,13.0,...,2125,745.0,8.0,803,753.0,-10.0,0,0,0,2015-01-01


In [14]:
# Saturday has the least flights
# Thursday has the most flights
flights_df.groupby('DAY_OF_WEEK').count()

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,DATE
DAY_OF_WEEK,,,,,,,,,,,,,,,,,,,,,
1,542361,542361,542361,542361,542361,542361,542361,542361,542361,542361,...,542361,542361,542361,542361,542361,542361,542361,542361,542361,542361
2,532251,532251,532251,532251,532251,532251,532251,532251,532251,532251,...,532251,532251,532251,532251,532251,532251,532251,532251,532251,532251
3,538787,538787,538787,538787,538787,538787,538787,538787,538787,538787,...,538787,538787,538787,538787,538787,538787,538787,538787,538787,538787
4,546723,546723,546723,546723,546723,546723,546723,546723,546723,546723,...,546723,546723,546723,546723,546723,546723,546723,546723,546723,546723
5,539599,539599,539599,539599,539599,539599,539599,539599,539599,539599,...,539599,539599,539599,539599,539599,539599,539599,539599,539599,539599
6,432562,432562,432562,432562,432562,432562,432562,432562,432562,432562,...,432562,432562,432562,432562,432562,432562,432562,432562,432562,432562
7,507778,507778,507778,507778,507778,507778,507778,507778,507778,507778,...,507778,507778,507778,507778,507778,507778,507778,507778,507778,507778


In [15]:
# Change Day of week from num to string
flights_df.loc[flights_df["DAY_OF_WEEK"] == 1, "DAY_OF_WEEK"] = "Monday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 2, "DAY_OF_WEEK"] = "Tuesday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 3, "DAY_OF_WEEK"] = "Wednesday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 4, "DAY_OF_WEEK"] = "Thursday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 5, "DAY_OF_WEEK"] = "Friday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 6, "DAY_OF_WEEK"] = "Saturday"
flights_df.loc[flights_df["DAY_OF_WEEK"] == 7, "DAY_OF_WEEK"] = "Sunday"

### Merge with airline and airport df

In [16]:
flights_air = pd.merge(flights_df,airline_df,left_on='AIRLINE',right_on='AIRLINE_ID',how='left')
flights_air = flights_air.drop(columns=['AIRLINE_ID'])
flights_air = flights_air.rename(columns={'AIRLINE_x': 'AIRLINE_ID', 'AIRLINE_y': 'AIRLINE'})
flights_air.head(5)

,DAY_OF_WEEK,AIRLINE_ID,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,DATE,AIRLINE
0,Thursday,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,...,737.0,4.0,750,741.0,-9.0,0,0,0,2015-01-01,American Airlines Inc.
1,Thursday,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,...,800.0,11.0,806,811.0,5.0,0,0,0,2015-01-01,US Airways Inc.
2,Thursday,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,...,748.0,8.0,805,756.0,-9.0,0,0,0,2015-01-01,American Airlines Inc.
3,Thursday,DL,806,N3730B,SFO,MSP,25,20.0,-5.0,18.0,...,604.0,6.0,602,610.0,8.0,0,0,0,2015-01-01,Delta Air Lines Inc.
4,Thursday,US,2013,N584UW,LAX,CLT,30,44.0,14.0,13.0,...,745.0,8.0,803,753.0,-10.0,0,0,0,2015-01-01,US Airways Inc.


### Delayed Flights

In [17]:
# The total df of delay flights
arrival_Delay = flights_air.loc[flights_air['ARRIVAL_DELAY'] > 0].reset_index(drop=True)  # negative value means arrive earlier
departure_Delay=flights_air.loc[flights_air['DEPARTURE_DELAY']> 0].reset_index(drop=True)

#  for dep_Delay: 2125618 rows
#  for arr_Delay: 2086896 rows

In [18]:
# Both arrival and departure are delayed
delay_Flights = flights_air.loc[(flights_air['ARRIVAL_DELAY'] > 0) & (flights_air['DEPARTURE_DELAY'] > 0)].reset_index(drop=True)
delay_Flights.head(5)
# 1508147 total

,DAY_OF_WEEK,AIRLINE_ID,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,DATE,AIRLINE
0,Thursday,AA,2392,N3HRAA,DEN,MIA,120,141.0,21.0,12.0,...,701.0,8.0,707,709.0,2.0,0,0,0,2015-01-01,American Airlines Inc.
1,Thursday,UA,1500,N30401,ORD,IAH,510,514.0,4.0,23.0,...,804.0,12.0,805,816.0,11.0,0,0,0,2015-01-01,United Air Lines Inc.
2,Thursday,AA,1057,N3ASAA,DFW,MIA,515,703.0,108.0,15.0,...,1031.0,7.0,856,1038.0,102.0,0,0,0,2015-01-01,American Airlines Inc.
3,Thursday,US,425,N174US,PDX,PHX,520,620.0,60.0,13.0,...,945.0,5.0,850,950.0,60.0,0,0,0,2015-01-01,US Airways Inc.
4,Thursday,AA,89,N3KVAA,IAH,MIA,520,618.0,58.0,19.0,...,928.0,7.0,841,935.0,54.0,0,0,0,2015-01-01,American Airlines Inc.


In [19]:
# Further Narrow down the dataframe
delay_Flights = delay_Flights[['AIRLINE_ID','AIRLINE','DATE','DAY_OF_WEEK','TAIL_NUMBER','FLIGHT_NUMBER','ORIGIN_AIRPORT',
                                     'DESTINATION_AIRPORT','DEPARTURE_DELAY','ARRIVAL_DELAY']]
delay_Flights.head(5)
# We are going to merge this dataset with tweets below.

,AIRLINE_ID,AIRLINE,DATE,DAY_OF_WEEK,TAIL_NUMBER,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY
0,AA,American Airlines Inc.,2015-01-01,Thursday,N3HRAA,2392,DEN,MIA,21.0,2.0
1,UA,United Air Lines Inc.,2015-01-01,Thursday,N30401,1500,ORD,IAH,4.0,11.0
2,AA,American Airlines Inc.,2015-01-01,Thursday,N3ASAA,1057,DFW,MIA,108.0,102.0
3,US,US Airways Inc.,2015-01-01,Thursday,N174US,425,PDX,PHX,60.0,60.0
4,AA,American Airlines Inc.,2015-01-01,Thursday,N3KVAA,89,IAH,MIA,58.0,54.0


In [20]:
delay_Flights.groupby('AIRLINE').count()

,AIRLINE_ID,DATE,DAY_OF_WEEK,TAIL_NUMBER,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY
AIRLINE,,,,,,,,,
American Airlines Inc.,172258,172258,172258,172258,172258,172258,172258,172258,172258
Delta Air Lines Inc.,175530,175530,175530,175530,175530,175530,175530,175530,175530
Southwest Airlines Co.,391005,391005,391005,391005,391005,391005,391005,391005,391005
US Airways Inc.,46519,46519,46519,46519,46519,46519,46519,46519,46519
United Air Lines Inc.,156697,156697,156697,156697,156697,156697,156697,156697,156697
Virgin America,16577,16577,16577,16577,16577,16577,16577,16577,16577


### Merge with airport_df to get the airport latitude & longitude

In [21]:
airports_df

,AIRPORT_ID,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [22]:
# Merge First for the Original (ORIG) Airport Info
flights_final = pd.merge(delay_Flights,airports_df,left_on='ORIGIN_AIRPORT',right_on='AIRPORT_ID',how='left')
flights_final = flights_final.drop(columns=['AIRPORT_ID','TAIL_NUMBER','FLIGHT_NUMBER','COUNTRY'])
flights_final = flights_final.rename(columns={'AIRPORT': 'ORIG_AIRPORT', 'CITY': 'ORIG_CITY', 
                                              'STATE':'ORIG_STATE','LATITUDE':'ORIG_LATITUDE','LONGITUDE':'ORIG_LONGITUDE'})
flights_final.head(5)

,AIRLINE_ID,AIRLINE,DATE,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY,ORIG_AIRPORT,ORIG_CITY,ORIG_STATE,ORIG_LATITUDE,ORIG_LONGITUDE
0,AA,American Airlines Inc.,2015-01-01,Thursday,DEN,MIA,21.0,2.0,Denver International Airport,Denver,CO,39.85841,-104.66700
1,UA,United Air Lines Inc.,2015-01-01,Thursday,ORD,IAH,4.0,11.0,Chicago O'Hare International Airport,Chicago,IL,41.97960,-87.90446
2,AA,American Airlines Inc.,2015-01-01,Thursday,DFW,MIA,108.0,102.0,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,32.89595,-97.03720
3,US,US Airways Inc.,2015-01-01,Thursday,PDX,PHX,60.0,60.0,Portland International Airport,Portland,OR,45.58872,-122.59750
4,AA,American Airlines Inc.,2015-01-01,Thursday,IAH,MIA,58.0,54.0,George Bush Intercontinental Airport,Houston,TX,29.98047,-95.33972


In [23]:
# Merge second for the Destination (DEST_) Airport Info
flights_final = pd.merge(flights_final,airports_df,left_on='DESTINATION_AIRPORT',right_on='AIRPORT_ID',how='left')
flights_final = flights_final.drop(columns=['AIRPORT_ID','COUNTRY'])
flights_final = flights_final.rename(columns={'AIRPORT': 'DEST_AIRPORT', 'CITY': 'DEST_CITY', 
                                              'STATE':'DEST_STATE','LATITUDE':'DEST_LATITUDE','LONGITUDE':'DEST_LONGITUDE'})
flights_final.head(5)

,AIRLINE_ID,AIRLINE,DATE,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY,ORIG_AIRPORT,ORIG_CITY,ORIG_STATE,ORIG_LATITUDE,ORIG_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_LATITUDE,DEST_LONGITUDE
0,AA,American Airlines Inc.,2015-01-01,Thursday,DEN,MIA,21.0,2.0,Denver International Airport,Denver,CO,39.85841,-104.66700,Miami International Airport,Miami,FL,25.79325,-80.29056
1,UA,United Air Lines Inc.,2015-01-01,Thursday,ORD,IAH,4.0,11.0,Chicago O'Hare International Airport,Chicago,IL,41.97960,-87.90446,George Bush Intercontinental Airport,Houston,TX,29.98047,-95.33972
2,AA,American Airlines Inc.,2015-01-01,Thursday,DFW,MIA,108.0,102.0,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,32.89595,-97.03720,Miami International Airport,Miami,FL,25.79325,-80.29056
3,US,US Airways Inc.,2015-01-01,Thursday,PDX,PHX,60.0,60.0,Portland International Airport,Portland,OR,45.58872,-122.59750,Phoenix Sky Harbor International Airport,Phoenix,AZ,33.43417,-112.00806
4,AA,American Airlines Inc.,2015-01-01,Thursday,IAH,MIA,58.0,54.0,George Bush Intercontinental Airport,Houston,TX,29.98047,-95.33972,Miami International Airport,Miami,FL,25.79325,-80.29056


# December Flights & More Analysis

In [24]:
# Only select Dec 
dec = (flights_final['DATE'] >= '2015-12-01') 
flights_dec = flights_final.loc[dec]
flights_dec.head(5)

,AIRLINE_ID,AIRLINE,DATE,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,ARRIVAL_DELAY,ORIG_AIRPORT,ORIG_CITY,ORIG_STATE,ORIG_LATITUDE,ORIG_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_LATITUDE,DEST_LONGITUDE
871006,DL,Delta Air Lines Inc.,2015-12-01,Tuesday,ANC,SEA,2.0,8.0,Ted Stevens Anchorage International Airport,Anchorage,AK,61.17432,-149.99619,Seattle-Tacoma International Airport,Seattle,WA,47.44898,-122.30931
871007,AA,American Airlines Inc.,2015-12-01,Tuesday,PHX,MSP,30.0,27.0,Phoenix Sky Harbor International Airport,Phoenix,AZ,33.43417,-112.00806,Minneapolis-Saint Paul International Airport,Minneapolis,MN,44.88055,-93.21692
871008,UA,United Air Lines Inc.,2015-12-01,Tuesday,BQN,EWR,33.0,9.0,Rafael Hernández Airport,Aguadilla,PR,18.49486,-67.12944,Newark Liberty International Airport,Newark,NJ,40.69250,-74.16866
871009,AA,American Airlines Inc.,2015-12-01,Tuesday,SFO,ORD,7.0,1.0,San Francisco International Airport,San Francisco,CA,37.61900,-122.37484,Chicago O'Hare International Airport,Chicago,IL,41.97960,-87.90446
871010,AA,American Airlines Inc.,2015-12-01,Tuesday,ORD,PHX,42.0,24.0,Chicago O'Hare International Airport,Chicago,IL,41.97960,-87.90446,Phoenix Sky Harbor International Airport,Phoenix,AZ,33.43417,-112.00806


In [25]:
flights_byAirline = flights_dec.groupby('AIRLINE').count()[['AIRLINE_ID']]
flights_byAirline = flights_byAirline.rename(columns = {'AIRLINE_ID':'TOTAL'})
flights_byAirline

,TOTAL
AIRLINE,
American Airlines Inc.,19088
Delta Air Lines Inc.,16101
Southwest Airlines Co.,38036
United Air Lines Inc.,12317
Virgin America,2038


In [26]:
flights_byDay = flights_dec.groupby('DAY_OF_WEEK').count()[['AIRLINE']].rename(columns = {'AIRLINE':'TOTAL'})
flights_byDay

,TOTAL
DAY_OF_WEEK,
Friday,9044
Monday,12093
Saturday,8296
Sunday,11604
Thursday,13274
Tuesday,15692
Wednesday,17577


## Count by state

In [27]:
flights_byState = flights_dec.groupby('ORIG_STATE').count()[['AIRLINE']].rename(columns ={'AIRLINE': 'TOTAL'})
flights_byState.head(10)

,TOTAL
ORIG_STATE,
AK,72
AL,193
AR,86
AZ,3781
CA,14506
CO,4148
CT,260
FL,7190
GA,6154


## Count By Date

In [28]:
flights_byDate = flights_dec.groupby('DATE').count()[['AIRLINE']].rename(columns ={'AIRLINE': 'TOTAL'})
flights_byDate.head(5)

,TOTAL
DATE,
2015-12-01,2745
2015-12-02,2726
2015-12-03,2265
2015-12-04,1478
2015-12-05,953


### Cancelled Flights

In [29]:
# Find out the values within the column
flights_air['CANCELLED'].value_counts()

0    3598101
1      41960
Name: CANCELLED, dtype: int64

In [30]:
cancelled_flights = flights_air.loc[flights_air['CANCELLED']==1].reset_index(drop=True)

In [31]:
# Count total number of cancelled flights
cancelled_flights.count()['CANCELLED']

41960

In [32]:
# Remove irrelevant columns 
cancelled_flights = cancelled_flights[['AIRLINE_ID','AIRLINE','DATE','DAY_OF_WEEK','ORIGIN_AIRPORT',
                                     'DESTINATION_AIRPORT']]
cancelled_flights.head(5)

,AIRLINE_ID,AIRLINE,DATE,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT
0,AA,American Airlines Inc.,2015-01-01,Thursday,PHX,DFW
1,AA,American Airlines Inc.,2015-01-01,Thursday,DCA,DFW
2,AA,American Airlines Inc.,2015-01-01,Thursday,LGA,DFW
3,AA,American Airlines Inc.,2015-01-01,Thursday,MCO,DFW
4,AA,American Airlines Inc.,2015-01-01,Thursday,SFO,DFW


In [33]:
dec = (cancelled_flights['DATE'] >= '2015-12-01') 
canc_flights_dec = cancelled_flights.loc[dec]
canc_flights_dec.head(5)

,AIRLINE_ID,AIRLINE,DATE,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT
38045,WN,Southwest Airlines Co.,2015-12-01,Tuesday,LAX,PHX
38046,WN,Southwest Airlines Co.,2015-12-01,Tuesday,LAS,PHX
38047,AA,American Airlines Inc.,2015-12-01,Tuesday,FLL,DCA
38048,AA,American Airlines Inc.,2015-12-01,Tuesday,DFW,LAX
38049,UA,United Air Lines Inc.,2015-12-01,Tuesday,OGG,SFO


In [34]:
canc_byDate = canc_flights_dec.groupby('DATE').count()[['AIRLINE_ID']].rename(columns = {'AIRLINE_ID':'CANC_TOTAL'})
canc_byDate.head(5)

,CANC_TOTAL
DATE,
2015-12-01,15
2015-12-02,43
2015-12-03,22
2015-12-04,10
2015-12-05,49


In [35]:
flights_byDate2 = pd.merge(flights_byDate,canc_byDate,left_on='DATE',right_on='DATE',how='left')
flights_byDate2.head(5)

,TOTAL,CANC_TOTAL
DATE,,
2015-12-01,2745,15
2015-12-02,2726,43
2015-12-03,2265,22
2015-12-04,1478,10
2015-12-05,953,49


In [36]:
canc_byAirline = cancelled_flights.groupby('AIRLINE').count()[['AIRLINE_ID']].rename(columns = {'AIRLINE_ID':'TOTAL'})
canc_byAirline.head(5)

,TOTAL
AIRLINE,
American Airlines Inc.,10919
Delta Air Lines Inc.,3824
Southwest Airlines Co.,16043
US Airways Inc.,4067
United Air Lines Inc.,6573


In [37]:
canc_ByDay = cancelled_flights.groupby('DAY_OF_WEEK').count()[['AIRLINE']].rename(columns = {'AIRLINE':'TOTAL'})
canc_ByDay

,TOTAL
DAY_OF_WEEK,
Friday,3736
Monday,9534
Saturday,4446
Sunday,5777
Thursday,5478
Tuesday,7702
Wednesday,5287


# 4. Tweets data

In [38]:
tweets_file = "./Resources/Tweets.csv"
tweets_df = pd.read_csv(tweets_file)
tweets_df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [39]:
tweets_df = tweets_df[['airline','airline_sentiment','tweet_created','text']]
tweets_df = tweets_df.rename(columns = {'tweet_created': 'tweeted_date', 'airline_sentiment': 'attitude'})
tweets_df.head(5)

,airline,attitude,tweeted_date,text
0,Virgin America,neutral,2015-02-24 11:35:52 -0800,@VirginAmerica What @dhepburn said.
1,Virgin America,positive,2015-02-24 11:15:59 -0800,@VirginAmerica plus you've added commercials t...
2,Virgin America,neutral,2015-02-24 11:15:48 -0800,@VirginAmerica I didn't today... Must mean I n...
3,Virgin America,negative,2015-02-24 11:15:36 -0800,@VirginAmerica it's really aggressive to blast...
4,Virgin America,negative,2015-02-24 11:14:45 -0800,@VirginAmerica and it's a really big bad thing...


In [40]:
tweets_df['airline'].value_counts()

United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: airline, dtype: int64

In [41]:
# Update the airline Name so it matches the flights_df
tweets_df.loc[tweets_df["airline"] == "American", "AIRLINE"] = "American Airlines Inc."
tweets_df.loc[tweets_df["airline"] == "Delta",    "AIRLINE"] = "Delta Air Lines Inc."
tweets_df.loc[tweets_df["airline"] == "Southwest","AIRLINE"] = "Southwest Airlines Co."
tweets_df.loc[tweets_df["airline"] == "US Airways","AIRLINE"] = "US Airways Inc."
tweets_df.loc[tweets_df["airline"] == "United", "AIRLINE"] = "United Air Lines Inc."
tweets_df.loc[tweets_df["airline"] == "Virgin America", "AIRLINE"] = "Virgin America"

In [42]:
# Clean the tweet_created column to get the Date
tweets_df['DATE'] = pd.to_datetime(tweets_df['tweeted_date'])
tweets_df['DATE']=pd.DatetimeIndex(tweets_df['DATE']).date
tweets_df = tweets_df.drop(columns=['tweeted_date','airline'])
tweets_df

,attitude,text,AIRLINE,DATE
0,neutral,@VirginAmerica What @dhepburn said.,Virgin America,2015-02-24
1,positive,@VirginAmerica plus you've added commercials t...,Virgin America,2015-02-24
2,neutral,@VirginAmerica I didn't today... Must mean I n...,Virgin America,2015-02-24
3,negative,@VirginAmerica it's really aggressive to blast...,Virgin America,2015-02-24
4,negative,@VirginAmerica and it's a really big bad thing...,Virgin America,2015-02-24
...,...,...,...,...
14635,positive,@AmericanAir thank you we got on a different f...,American Airlines Inc.,2015-02-22
14636,negative,@AmericanAir leaving over 20 minutes Late Flig...,American Airlines Inc.,2015-02-22
14637,neutral,@AmericanAir Please bring American Airlines to...,American Airlines Inc.,2015-02-22
14638,negative,"@AmericanAir you have my money, you change my ...",American Airlines Inc.,2015-02-22


### Save to json file *optional

In [ ]:
# tweets_df.to_json(r'Resources/output/tweets_df.json',orient ='records') 

In [43]:
tweets_df['DATE'] = pd.to_datetime(tweets_df['DATE'])
tweets_df.dtypes

attitude            object
text                object
AIRLINE             object
DATE        datetime64[ns]
dtype: object

In [ ]:
# Another Method:
# tweets_df['DATE'] = tweets_df['tweet_created'].str[0:10]
# tweets_df['DATE'] = pd.to_datetime(tweets_df['DATE'])
# tweets_df.head()
# tweets_df = tweets_df.drop(columns=['tweet_created'])
# tweets_df.dtypes

In [44]:
tweets_df['AIRLINE'].value_counts()

United Air Lines Inc.     3822
US Airways Inc.           2913
American Airlines Inc.    2759
Southwest Airlines Co.    2420
Delta Air Lines Inc.      2222
Virgin America             504
Name: AIRLINE, dtype: int64

In [45]:
tweets_df['attitude'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: attitude, dtype: int64

In [46]:
tweets_df.groupby(['AIRLINE','attitude']).count()
# Apparently, United AirLines has the most negative reviews
# Virgin America has the least nagative reviews

text  DATE
AIRLINE                attitude            
American Airlines Inc. negative  1960  1960
                       neutral    463   463
                       positive   336   336
Delta Air Lines Inc.   negative   955   955
                       neutral    723   723
                       positive   544   544
Southwest Airlines Co. negative  1186  1186
                       neutral    664   664
                       positive   570   570
US Airways Inc.        negative  2263  2263
                       neutral    381   381
                       positive   269   269
United Air Lines Inc.  negative  2633  2633
                       neutral    697   697
                       positive   492   492
Virgin America         negative   181   181
                       neutral    171   171
                       positive   152   152

In [ ]:
# If we want to change NaN valule to 0
# tweets_df.loc[tweets_df['set_of_numbers'].isnull(),'value_is_NaN'] = 'Yes'
# tweets_df.loc[tweets_df['set_of_numbers'].notnull(),'value_is_NaN'] = 'No'

# count_nan = tweets_df.loc[tweets_df['value_is_NaN']=='Yes'].count()
# print (count_nan)

## Output to csv & JSON *(optional)

In [ ]:
flights_dec.to_csv("Resources/output/holiday.csv", index=True)

flights_dec.to_json(r'Resources/output/holiday.json',orient ='records') 
flights_byAirline.to_json(r'Resources/output/flights_ByAirline.json',orient = 'columns')
flights_byDay.to_json(r'Resources/output/flights_ByDay.json',orient = 'columns')

flights_byState.to_json(r'Resources/output/flights_byState.json',orient = 'columns')
flights_byDate2.to_json(r'Resources/output/flights_byDate.json',orient = 'columns')
tweets_df.to_json(r'Resources/output/tweets_df.json',orient = 'columns')

# Connect to MongoDB for project 2

In [47]:
import pymysql
pymysql.install_as_MySQLdb()
import pymongo
import MySQLdb

In [48]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.flights2_db

In [49]:
flights_byState = flights_byState.reset_index()
flights_byDate = flights_byDate2.reset_index()
flights_byAirline = flights_byAirline.reset_index()
flights_byDay = flights_byDay.reset_index()

In [50]:
# Insert smaller dataframes
db.flights_ByAirline.insert_many(flights_byAirline.to_dict('records'))
db.flights_byDay.insert_many(flights_byDay.to_dict('records'))
db.flights_byState.insert_many(flights_byState.to_dict('records'))
db.flights_byDate.insert_many(flights_byDate.to_dict('records'))

In [51]:
for index,row in flights_dec.iterrows():
    db.flightDec.insert_one(dict(row))

In [52]:
# Insert tweets data
db.tweets_df.insert_many(tweets_df.to_dict('records'))